In [1]:
# evaluate_openrouter.py
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils.openrouter_deepeval_llm import OpenRouterDeepevalLLM
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric

# Initialize custom LLM
llm = OpenRouterDeepevalLLM()

# Test case
test_case = LLMTestCase(
    input="Explain how photosynthesis works.",
    expected_output="Plants use sunlight to convert carbon dioxide and water into glucose and oxygen.",
    actual_output=None # you might generate that
)

# Let's generate actual_output
test_case.actual_output = llm.generate(test_case.input)

# Choose metric(s)
metric = AnswerRelevancyMetric(model=llm, threshold=0.7, include_reason=True)

score = metric.measure(test_case)
print("Score:", score)
print("Reason:", metric.reason if hasattr(metric, "reason") else None)


Output()

AttributeError: 'str' object has no attribute 'statements'

In [6]:
from deepeval import assert_test
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval
from dotenv import load_dotenv
from src.utils.ilm_client import OpenRouterILMClient  # 👈 use your client
import os

# Load .env
load_dotenv()

# Initialize OpenRouter LLM client
ilm_client = OpenRouterILMClient()  # automatically loads API key & model from .env

def test_sample_1():
    # Define metric using our client
    correctness_metric = GEval(
        name="correctness",
        criteria="Does the response correctly answer the question based on the provided context?",
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
        threshold=0.5,
        model=ilm_client  # 👈 use our OpenRouterILMClient
    )

    # Input data
    input_data = {"question": "What is 2 + 2?"}
    
    # Get actual output from OpenRouter
    actual_output = ilm_client.generate(input_data["question"])
    print(f"Actual Output: {actual_output}")
    
    # Build LLMTestCase
    test_case = LLMTestCase(
        name="Sample Test Case 1",
        description="Test case to evaluate the correctness of a simple addition operation.",
        input=input_data["question"],                # the prompt
        expected_output="4",                          # expected answer
        actual_output=actual_output.strip(),         # remove leading/trailing whitespace
        metrics=[correctness_metric]
    )

    # Run assertion
    assert_test(test_case, [correctness_metric])

    
test_sample_1()

    

TypeError: Unsupported type for model: <class 'src.utils.ilm_client.OpenRouterILMClient'>. Expected None, str, DeepEvalBaseLLM, GPTModel, AzureOpenAIModel, LiteLLMModel, OllamaModel, LocalModel.

In [11]:
from deepeval import assert_test
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval
from deepeval.models.llms import LiteLLMModel
from dotenv import load_dotenv
import os

# Load env
load_dotenv("../.env")

# Use LiteLLMModel for OpenRouter
llm = LiteLLMModel(model="openrouter/nvidia/nemotron-nano-9b-v2:free")

def test_sample_1():
    correctness_metric = GEval(
        name="correctness",
        criteria="Does the response correctly answer the question?",
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
        threshold=0.5,
        model=llm
    )

    question = "What is 2 + 2?"
    actual_output = llm.generate(question)  # uses OpenRouter under the hood

    test_case = LLMTestCase(
        name="Sample Test Case 1",
        description="Simple addition test.",
        input=question,
        expected_output="4",
        actual_output=actual_output[0].strip(),
        metrics=[correctness_metric]
    )

    assert_test(test_case, [correctness_metric])

test_sample_1()

Output()